# SITCOM-1136 - M1M3 - analyze position and rotation stability throughout a tracking period

2023-12-14 Laura

This notebook evaluates displacements of M1M3 for X, Y, Z, RX, RY, and RZ during the period beteween slews while tracking (approximately 30 s) and check if they comply with the 2 micron and 2e-5 degree requirement.


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from astropy.time import Time, TimezoneInfo
from statsmodels.tsa.stattools import adfuller

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient

from datetime import timedelta, datetime
import warnings
warnings.filterwarnings('ignore')

from scipy import stats 

In [ ]:
# create a client to retrieve datasets in the EFD database
client = makeEfdClient()

### Define relevant settings

#### Requirements

In [ ]:
req_rms_position = 2e-3 ## mm, tolerance from repeatability requirement for IMS positional
req_rms_rotation = 2e-5 ## degrees, tolerance from repeatability requirement for IMS rotational

### Define functions

In [ ]:
def computeSettleTrack(
    df_ims,  # input data frame
    tt_start="2023-06-01T06:00:0Z",  # time for slew start
    tt_end = "2023-06-01T06:00:0Z",  # time for track stop
    imsColumn="xPosition",  # IMS column
    rmsReq=2e-3,  # requirement in appropriate units
    delta_t=5,
    seqNum=000
):
    if "Position" in imsColumn:
        units = "mm"
        ylimMax = rmsReq + 0.005
    elif "Rotation" in imsColumn:
        units = "deg"
        ylimMax = rmsReq + 0.00005
    else:
        print("Unidentified column")
        return -1
 
    # Define Times
    T0 = pd.to_datetime(tt_start) - pd.to_timedelta(delta_t, unit="s")
    T1 = pd.to_datetime(tt_end) + pd.to_timedelta(delta_t, unit="s")
    t_track_starts = pd.to_datetime(tt_start)
    t_track_ends = pd.to_datetime(tt_end)
    # We removed 0.1s from the tracking (to eliminate the moment when the telescope starts moving) in order to calculate de RMS in the start and end of the tracking.
    # The RMS is calculated with values close to a specific time of the tracking, in this way we avoid taking velues from the slew
    t_track_starts_check = pd.to_datetime(tt_start) + pd.to_timedelta(0.1, unit="s")
    t_track_ends_check = pd.to_datetime(tt_end) - pd.to_timedelta(0.1, unit="s")

    # Define Target
    # targetVariablePlot takes the data frame for the complete plot range
    targetVariablePlot = df_ims[imsColumn][T0 : T1]
    
    # Define index
    idxT0 = df_ims.index[  # index in dataframe closest in time to start of plot
        df_ims.index.get_indexer([pd.to_datetime(t_track_starts)+ pd.to_timedelta(10, unit="s")], method="nearest")
    ]
    idxT1 = df_ims.index[  # index in dataframe closest in time to + pd.to_timedelta(delta_t, unit="s")end of plot
            df_ims.index.get_indexer([pd.to_datetime(t_track_ends)- pd.to_timedelta(10, unit="s")], method="nearest")
    ] 

    targetVariableReference = [
        float(df_ims[imsColumn][idxT0]),
        float(df_ims[imsColumn][idxT1]),
    ]
    if len(targetVariablePlot.index) == 0:
        print("Data frame is empty")
        return -1
    
    correctedVariablePlot = targetVariablePlot - targetVariableReference[1]

    # Tracking time
    tts_unix = Time(t_track_starts).unix
    tte_unix = Time(t_track_ends).unix
    seconds = tte_unix - tts_unix
    seconds_decimal = round(seconds,2)

    # Check stability
    filtered_correctedVariablePlot = correctedVariablePlot.loc[t_track_starts_check:t_track_ends_check]
    
    # rms
    rolling = 10  # 50 is approx. 1 s
    rms = filtered_correctedVariablePlot.rolling(rolling).std()
    
    # Plot
    title = f"Tracking time: {seconds_decimal} seconds. SeqNum:" '{:.2f}'.format(seqNum)
    fig = plt.figure()
    label = "Corrected " + imsColumn + "(" + units + ") difference wrt end of plot"
    plt.title(title)    
    
    plt.plot(
            correctedVariablePlot,
            color="red",
            ls="dashed",
            lw="0.5",
            label=label,
    )

    plt.plot(rms, lw=1.2, c='blue', label="RMS") 
        
    plt.axhline(-rmsReq, lw="0.75", c="k", ls="dashed", label=f"IMS repeatability req.")
    plt.axhline(rmsReq, lw="0.75", c="k", ls="dashed")
    plt.axvline(x=t_track_starts, color='black', linestyle='dashed', linewidth=1.2, label='Start of Tracking')
    plt.axvline(x=t_track_ends, color='black', linestyle='dashed', linewidth=1.2, label='End of Tracking')
    plt.axvspan(t_track_starts, t_track_starts + pd.to_timedelta(3, unit="s"), color='lightblue', alpha=0.5, label='settling time? 3s')

    division = seconds / 5.
    plt.xticks([t_track_starts, t_track_starts + pd.to_timedelta(division, unit="s") ,
                t_track_starts + pd.to_timedelta(2*division, unit="s"),
                t_track_starts + pd.to_timedelta(3*division, unit="s"),
                t_track_starts + pd.to_timedelta(4*division, unit="s"),
                t_track_ends], 
               [0, round(division,2), round(2*division,2), round(3*division,2) , round(4*division,2),round((tte_unix - tts_unix), 2)])


    
    plt.xlabel("Time [UTC]")
    plt.ylabel(f"{imsColumn} {units}")
    plt.ylim(-ylimMax, ylimMax)
    fig.autofmt_xdate()
    plt.legend(loc="upper right", fontsize="8")
    fig.tight_layout()


In [ ]:
def detectUnstableEvents(
    df_ims,  # input data frame
    tt_start="2023-06-01T06:00:0Z",  # time for slew start
    tt_end = "2023-06-01T06:00:0Z",  # time for track stop
    imsColumn="xPosition",  # IMS column
    rmsReq=2e-3,  # requirement in appropriate units
    delta_t=5
):
    if "Position" in imsColumn:
        units = "mm"
        ylimMax = rmsReq + 0.005
    elif "Rotation" in imsColumn:
        units = "deg"
        ylimMax = rmsReq + 0.00005
    else:
        print("Unidentified column")
        return -1
 
    # Define Times
    T0 = pd.to_datetime(tt_start) - pd.to_timedelta(delta_t, unit="s")
    T1 = pd.to_datetime(tt_end) + pd.to_timedelta(delta_t, unit="s")
    t_track_starts = pd.to_datetime(tt_start)
    t_track_ends = pd.to_datetime(tt_end)
    # We removed 0.1s from the tracking check to eliminate the moment when the telescope starts moving.
    t_track_starts_check = pd.to_datetime(tt_start) + pd.to_timedelta(0.1, unit="s")
    t_track_ends_check = pd.to_datetime(tt_end) - pd.to_timedelta(0.3, unit="s")


    
    # Define Target
    targetVariablePlot = df_ims[imsColumn][T0 : T1]
    
    # Define index
    idxT0 = df_ims.index[  # index in dataframe closest in time to start of plot
        df_ims.index.get_indexer([pd.to_datetime(t_track_starts)+ pd.to_timedelta(10, unit="s")], method="nearest")
    ]
    idxT1 = df_ims.index[  # index in dataframe closest in time to + pd.to_timedelta(delta_t, unit="s")end of plot
            df_ims.index.get_indexer([pd.to_datetime(t_track_ends)- pd.to_timedelta(10, unit="s")], method="nearest")
    ] 

    targetVariableReference = [
        float(df_ims[imsColumn][idxT0]),
        float(df_ims[imsColumn][idxT1]),
    ]
    if len(targetVariablePlot.index) == 0:
        print("Data frame is empty")
        return -1
    
    correctedVariablePlot = targetVariablePlot - targetVariableReference[1]

    tts_unix = Time(t_track_starts).unix
    tte_unix =  Time(t_track_ends).unix
    time = tte_unix - tts_unix

    
    if time < 2:
        #print("Warning: tracking duration less than 2 seconds")
        return -1
        
    # Check stability
    filtered_correctedVariablePlot = correctedVariablePlot.loc[t_track_starts_check:t_track_ends_check]
    
    # rms
    rolling = 10  # 50 is approx. 1 s
    rms = filtered_correctedVariablePlot.rolling(rolling).std()
    
    for valor in rms:
      if valor > rmsReq:
        #print("Warning! Telescope movement during tracking exceeds the required rms limits")
        return -2
        break  
     
    return +1

### Definition of case

#### Observation day

In [ ]:
dayObs = 20231220  #20231129 20231222 

In [ ]:
## Define column names
# To study all particular cases comment and uncommet the position and rotation
all_columns = [
    "xPosition",
    #"yPosition",
    #"zPosition",
    #"xRotation",
    #"yRotation",
    #"zRotation",
]
pos_columns = [c for c in all_columns if "Position" in c]
rot_columns = [c for c in all_columns if "Rotation" in c]

In [ ]:
delta_t = 5

#### Get slew stops

In [ ]:
# Select data from a given date
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)

# Get lists of slew and track events
slews = [e for e in events if e.type==TMAState.SLEWING]
tracks = [e for e in events if e.type==TMAState.TRACKING]
print(f'Found {len(slews)} slews and {len(tracks)} tracks')

In [ ]:
# Get events related to soak tests (block 146 currently)
block146Events = []
for event in events:
    blockInfos = event.blockInfos
    if blockInfos is None:
        continue  # no block info attached to event at all

    # check if any of the attached blockInfos are for block 146
    blockNums = {b.blockNumber for b in blockInfos}
    if 146 in blockNums:
        block146Events.append(event)

print(f"Of the {len(events)} events, {len(block146Events)} relate to block 146.")

In [ ]:
# Print out sequential number of events that have certain characteristics
t = 0
tracks_block146 = []
for i in range(len(block146Events)):
    if (
        block146Events[i].endReason == TMAState.SLEWING
        and block146Events[i].type == TMAState.TRACKING
    ):
        #print(block146Events[i].seqNum, " ", block146Events[i].duration)
        t = t + 1
        print(block146Events[i].seqNum, end=", ")
        tracks_block146.append(block146Events[i])

In [ ]:
# Number of TRACKING in block 146
print(f"Of the {len(block146Events)} events of block 146, {t} are trackings.")

In [ ]:
## Time of the tracking:

# Get durations of the tracking
durations = [track.duration for track in tracks_block146]

# Calculate mean, standard deviation, and variance
mean_duration = np.mean(durations)
standard_deviation = np.std(durations)
variance = np.var(durations)
median_duration = np.median(durations)

# Round the durations for mode calculation
rounded_durations = [round(track.duration) for track in tracks_block146]

# Calculate mode of rounded durations
mode_duration = stats.mode(rounded_durations)


# Print the results
print("Number of trackings:", len(durations))
print("Mean duration of tracking:", mean_duration)
print("Median duration of tracking:", median_duration)
print("Mode of tracking duration (rounded):", mode_duration.mode[0])
print("Standard deviation of tracking duration:", standard_deviation)
print("Variance of tracking duration:", variance)
print("Maximum duration of tracking:", np.max(durations))
print("Minimum duration of tracking:", np.min(durations))


# Create a boxplot using seaborn
plt.figure(figsize=(8, 6))
sns.boxplot(y=durations, width=0.5)
plt.title('Boxplot of Tracking Duration')
plt.ylabel('Duration (s)')
plt.show()

# Create a boxplot using seaborn (zoom)
plt.figure(figsize=(8, 6))
sns.boxplot(y=durations, width=0.5)
plt.ylim(np.percentile(durations, 7), np.percentile(durations, 93))
plt.title('Zoomed Boxplot of Tracking Duration')
plt.ylabel('Duration (s)')
plt.show()


In [ ]:
unstable_events = {}
outRMSLimits_event = []
for event in enumerate(tracks_block146):
    targetSeqNum = event[1].seqNum
    is_unstable = False
    unstable_reason = "stable"
    #print(targetSeqNum)
    for t, tl in enumerate(tracks):
        if tl.seqNum == targetSeqNum:
            #print("tracking number seq Num: ", targetSeqNum)
            i_track = t

            # Select the information during the tracking
            t0 = Time(tracks[i_track].begin, format="isot", scale="utc")
            t0 = pd.to_datetime(t0.value, utc=True)  # astropy Time to Timestamp conversion
            t1 = Time(tracks[i_track].end, format="isot", scale="utc")
            t1 = pd.to_datetime(t1.value, utc=True)  # astropy Time to Timestamp conversion
            #print("Tracking stop at:", t1)
            
            # Get IMS data
            df_ims = getEfdData(
                     client, "lsst.sal.MTM1M3.imsData", event=tracks[i_track], postPadding=delta_t, prePadding=delta_t)
            df_ims = df_ims[all_columns]
            # Convert meter to milimeter
            df_ims[pos_columns] = df_ims[pos_columns] * 1e3
            
            #stability throughout a tracking period
            %matplotlib inline
            
            settle_intervals = np.empty(6)
            c = 0.

            for col in all_columns:
                if col in pos_columns:
                   req = req_rms_position
                else:
                   req = req_rms_rotation

                Unstable = detectUnstableEvents(
                           df_ims=df_ims,
                           tt_start=t0,
                           tt_end=t1,        
                           imsColumn=col,
                           rmsReq=req,
                           delta_t=5
                           )
                
                if Unstable == -1:
                   is_unstable = True
                   unstable_reason = "sortTime"
                   break
                if Unstable == -2:
                   is_unstable = True
                   unstable_reason = "outRMSLimits"
                   outRMSLimits_event.append(targetSeqNum)
                   break

    unstable_events[targetSeqNum] = {"is_unstable": is_unstable, "reason": unstable_reason}  

    if unstable_reason == "outRMSLimits": #"outRMSLimits":
        
        for col in all_columns:
                if col in pos_columns:
                   req = req_rms_position
                else:
                   req = req_rms_rotation 
                    
                settle_interval = computeSettleTrack(
                   df_ims=df_ims,
                   tt_start=t0,
                   tt_end=t1,        
                   imsColumn=col,
                   rmsReq=req,
                   #req_delta_t=req_delta_t,
                   #chi2prob=0.99,
                   delta_t=5,
                   seqNum=targetSeqNum
                   )   
                

In [ ]:
count_is_unstable = {"True": sum(event["is_unstable"] for event in unstable_events.values()),
               
                     "False": sum(not event["is_unstable"] for event in unstable_events.values())}
print(count_is_unstable)

In [ ]:
from collections import Counter
count_reason = [event["reason"] for event in unstable_events.values()]

print(Counter(count_reason))

## Tracking duration

In [ ]:
dayObs = 20231220

# Select data from a given date
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)

# Get lists of slew and track events
slews = [e for e in events if e.type==TMAState.SLEWING]
tracks = [e for e in events if e.type==TMAState.TRACKING]
print(f'Found {len(slews)} slews and {len(tracks)} tracks')

# Get events related to soak tests (block 146 currently)
block146Events = []
for event in events:
    blockInfos = event.blockInfos
    if blockInfos is None:
        continue  # no block info attached to event at all

    # check if any of the attached blockInfos are for block 146
    blockNums = {b.blockNumber for b in blockInfos}
    if 146 in blockNums:
        block146Events.append(event)

print(f"Of the {len(events)} events, {len(block146Events)} relate to block 146.")

# Print out sequential number of events that have certain characteristics
t = 0
tracks_block146 = []
for i in range(len(block146Events)):
    if (
        block146Events[i].endReason == TMAState.SLEWING
        and block146Events[i].type == TMAState.TRACKING
    ):
        print(block146Events[i].seqNum, " ", block146Events[i].duration)
        t = t + 1
        #print(block146Events[i].seqNum, end=", ")
        tracks_block146.append(block146Events[i])

# Number of TRACKING in block 146
print(f"Of the {len(block146Events)} events of block 146, {t} are trackings.")